In [ ]:
#default_exp filter
%load_ext autoreload
%autoreload 2

# Filtragem
> Este módulo concentra funções auxiliares específicas que filtram os dados do banco com campos e formatação de interesse para aplicações específicas como o [appAnalise](https://github.com/EricMagalhaesDelgado/appAnalise) por exemplo.

In [ ]:
#export
from pathlib import Path
import pandas as pd
from datetime import datetime
from anateldb.parser import *
from anateldb.query import *
from fastcore.test import *

In [ ]:
#export 
def filter_database(path):
    path = Path(path)
    dest = path / 'FiltroAppAnalise'
    dest.mkdir(parents = True, exist_ok=True)
    for p in dest.iterdir():
        if p.is_file():
            p.unlink()
    time = datetime.today().strftime("%Y%m%d_%H%M")
    mosaico = read_mosaico(path)
    stel = read_stel(path).sort_values('Frequência').loc[:, TELECOM]
    radcom = read_radcom(path)
    radcom['Serviço'] = '231'
    radcom['Status'] = 'RADCOM'
    radcom['Classe'] = radcom.Fase + '-' + radcom.Situação
    radcom = radcom.loc[:, RADIODIFUSAO]
    mosaico = mosaico.loc[:, RADIODIFUSAO]
    radiodifusao = mosaico.append(radcom)
#     radiodifusao['Frequência'] = radiodifusao['Frequência'].str.replace(',','.')
#     radiodifusao['Frequência'] = radiodifusao.Frequência.astype('float32')
    radiodifusao = radiodifusao.sort_values('Frequência').reset_index(drop=True)
    slma = (stel.Frequência >= 108) & (stel.Frequência <= 137)
    telecom = stel[~slma].reset_index(drop=True)
    slma = stel[slma].reset_index(drop=True)
#     radiodifusao = radiodifusao[(radiodifusao.Latitude != '') & (radiodifusao.Longitude != '')].reset_index(drop=True)
#     radiodifusao['Latitude'] = radiodifusao.Latitude.str.replace(',', '.').astype('float32')
#     radiodifusao['Longitude'] = radiodifusao.Longitude.str.replace(',', '.').astype('float32')

#     for c in radiodifusao.columns:
#         if c in ('Latitude', 'Longitude'):
#             radiodifusao[c] = radiodifusao[c].astype('float32', errors='ignore')
#         else:
#             radiodifusao[c] = radiodifusao[c].astype('str')
#     for c in slma.columns:
#         if c in ('Latitude', 'Longitude'):
#             slma[c] = slma[c].astype('float32', errors='ignore')
#         else:
#             slma[c] = slma[c].astype('str')
#     for c in telecom.columns:
#         if c in ('Latitude', 'Longitude'):
#             telecom[c] = telecom[c].astype('float32', errors='ignore')
#         else:
#             telecom[c] = telecom[c].astype('str')

    radiodifusao['Descrição'] = radiodifusao.Status + ', ' + radiodifusao.Classe + ', ' + radiodifusao.Entidade.str.title() + ' (' + \
                                radiodifusao.Fistel + ', ' + radiodifusao['Número da Estação'] + '), ' + radiodifusao.Município + '/' + \
                                radiodifusao.UF
    slma['Descrição'] = slma.Serviço + ', ' + slma.Entidade.str.title() + ' (' + slma.Fistel + ', ' +  slma['Número da Estação'] + '), ' + \
                        slma['Município'] + '/' + slma.UF
    telecom['Descrição'] = telecom.Serviço + ', ' + telecom.Entidade.str.title() + ' (' + telecom.Fistel + ', ' + telecom['Número da Estação'] + '), ' + \
                           telecom['Município'] + '/' + telecom.UF

    export_columns = ['Frequência', 'Latitude', 'Longitude', 'Descrição']
    radiodifusao = radiodifusao.loc[:, export_columns]
    slma = slma.loc[:, export_columns]
    telecom = telecom.loc[:, export_columns]
    radiodifusao.to_csv(f'{dest}/radiodifusao_{time}.csv', index=False)
    slma.to_csv(f'{dest}/108-137_{time}.csv', index=False)
    telecom.to_csv(f'{dest}/telecom_{time}.csv', index=False)
    radiodifusao.to_feather(f'{dest}/radiodifusao_{time}.fth')
    slma.to_feather(f'{dest}/108-137_{time}.fth')
    telecom.to_feather(f'{dest}/telecom_{time}.fth')
    with pd.ExcelWriter(f'{dest}/Base_de_Dados_{time}.xlsx') as workbook:
        radiodifusao.to_excel(workbook, sheet_name='Radiodifusão', index=False)
        slma.to_excel(workbook, sheet_name='108-137', index=False)
        telecom.to_excel(workbook, sheet_name='Telecom', index=False)

In [ ]:
path = 'D:\OneDrive - ANATEL\GR01FI3\BaseDados'

In [ ]:
mosaico = read_mosaico(path)
radcom = read_radcom(path)    

In [ ]:
radcom

,Frequência,Fase,Situação,Unidade,Entidade,Fistel,Número da Estação,Município,UF,Latitude,Longitude,CNPJ
0,104.9,3,A,MHz,ACADEMIA CULTURAL DE SANTA HELENA - ACULT - ST...,50011685115,641168764,Santa Helena,PR,-24.861389,-54.334722,00104477000117
1,87.9,3,M,MHz,ASSOCIACAO DOS MORADORES E PRODUT. RURAIS DE A...,50012524409,682699349,Assunção,PB,-7.074444,-36.731111,00284576000128
2,87.9,3,H,MHz,ASSOCIACAO BONFIM ESPERANCA- ABESPE,50011824689,659028590,Bonfim,MG,-20.323611,-44.246944,00575697000129
3,104.9,3,B,MHz,ASSOCIACAO DOS TRABALHADORES DE GUIMARANIA (ATG),50011398132,631410937,Guimarânia,MG,-18.843611,-46.792778,00792795000118
4,87.9,3,M,MHz,FUNDACAO ASSISTENCIAL LAR DA PAZ - FALP,50011398990,631412301,Dores do Indaiá,MG,-19.466667,-45.600000,00794510000188
...,...,...,...,...,...,...,...,...,...,...,...,...
4639,87.9,1,K,MHz,ASSOCIACAO AMIGOS DA CULTURA E DO MEIO AMBIENT...,50411347829,1011036964,Palmas,TO,-10.311667,-48.162222,08931976000190
4640,104.9,1,A,MHz,ASSOCIACAO RADIO COMUNITARIA MONTE SANTO FM,50416345301,1011037472,Monte Santo do Tocantins,TO,-10.005000,-48.988889,19001721000144
4641,104.9,P,M,MHz,ASSOCIACAO RADIO COMUNITARIA TOP FM,50416480004,1011044797,Araguatins,TO,-5.586389,-48.061111,19332116000156
4642,98.3,2,K,MHz,ASSOCIACAO DE RADIODIFUSAO CIDADE DE CRUZ ALTA,50416390609,1011044940,Cruz Alta,RS,-28.682222,-53.610278,97538346000180


In [ ]:
mosaico.head()

,Serviço,Status,Entidade,Fistel,CNPJ,Município,UF,Id,Número da Estação,Classe,Frequência,Latitude,Longitude,Num_Ato,Órgao,Data_Ato
0,TV,TV-C3,TELEVISAO OESTE BAIANO LTDA,06030116240,16395923000120,Barreiras,BA,57dbaad0dc4e3,322647029,A,79,"-12,102222222222","-44,994444444444",6245,ORLE,2019-10-08 03:30:03
1,TV,TV-C3,TV CABRALIA LTDA,06020354903,13494265000135,Itabuna,BA,57dbaad0ef8af,322623537,B,177,"-14,78","-39,260833333333",7613,ORLE,2018-10-10 13:21:26
2,TV,TV-C7,FUNDACAO FUNDESUL,50011828080,04188244000109,Porto Seguro,BA,57dbaad1077a6,637062230,C,515,"-16,353055555555","-39,386111111111",33606,ER,2003-02-06 00:00:00
3,TV,TV-C3,TELEVISAO SUL BAHIA DE TEIXEIRA DE FREITAS LTDA,06020355209,13985114000180,Teixeira de Freitas,BA,57dbaad132559,322623561,B,79,"-17,520833333333","-39,729444444444",9872,ORLE,2018-12-14 10:32:05
4,TV,TV-C7,FUNDACAO VALE DO JAGUARIBE,50012058165,04704007000145,Aracati,CE,57dbaad13efce,637069110,C,177,"-4,5666666666667","-37,77",32627,ER,2003-01-03 00:00:00


In [ ]:
from nbdev.export import notebook2script; notebook2script()

Converted filter.ipynb.
Converted index.ipynb.
Converted parser.ipynb.
Converted queries.ipynb.
